In [1]:
import requests
from langchain.llms.base import LLM
from typing import Optional, List


class MistralLLM(LLM):
    api_key: str
    model_name: str
    api_url: str = "https://api.mistral.ai/v1/chat/completions"

    @property
    def _llm_type(self) -> str:
        return "mistral"

    def _call(self, system_prompt: str, user_prompt: str,
              stop: Optional[List[str]] = None, max_tokens: int = 150, **kwargs) -> str:

        headers = {
            "Authorization": f"Bearer {self.api_key}",
            "Content-Type": "application/json"
        }
        payload = {
            "model": self.model_name,
            "max_tokens": max_tokens,
            "messages": [
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ],
            # "temperature": kwargs.get("temperature", 1),
            # "top_p": kwargs.get("top_p", 1),
            # "stream": kwargs.get("stream", False),
            # "stop": stop or ["string"],
            # "random_seed": kwargs.get("random_seed", 0),
            # "response_format": {"type": "text"},
            # "tools": kwargs.get("tools", []),
            # "tool_choice": kwargs.get("tool_choice", "auto"),
            # "presence_penalty": kwargs.get("presence_penalty", 0),
            # "frequency_penalty": kwargs.get("frequency_penalty", 0),
            # "n": kwargs.get("n", 1),
            # "safe_prompt": kwargs.get("safe_prompt", False)
        }

        response = requests.post(self.api_url, headers=headers, json=payload)
        response_data = response.json()

        if response.status_code != 200:
            raise Exception(f"Mistral API Error: {response_data.get('message', 'Unknown error')}")

        return response_data.get("choices", [{}])[0].get("message", {}).get("content", "")

    def generate(self, system_prompt: str, user_prompt: str, **kwargs) -> str:
        return self._call(system_prompt, user_prompt, **kwargs)

In [2]:
from langchain import LLMChain, PromptTemplate
from dotenv import load_dotenv
import os

load_dotenv('../.env')

MISTRAL_API_KEY = os.getenv("MISTRAL_API_KEY")

mistral_llm = MistralLLM(api_key=MISTRAL_API_KEY,
                         model_name="mistral-large-latest")


system_template = "You are an assistant that helps with translations."
user_template = "{input_text}"

system_prompt_template = PromptTemplate(input_variables=[], template=system_template)
user_prompt_template = PromptTemplate(input_variables=["input_text"], template=user_template)

user_input = "Придумай мем про Штирлица и машинное обучение"

formatted_system_prompt = system_prompt_template.format()
formatted_user_prompt = user_prompt_template.format(input_text=user_input)


response = mistral_llm.generate(system_prompt=formatted_system_prompt,
                                user_prompt=formatted_user_prompt,
                                max_tokens=512)
print(response)

Конечно! Вот идея для мема про Штирлица и машинное обучение:

**Изображение:** Штирлиц сидит за столом, окружённый кучей бумаг и книг, на экране компьютера виден график обучения модели.

**Текст:**

**Верхний текст:** "Когда Штирлиц пытается объяснить Мюллеру, почему его модель машинного обучения не может предсказать поведение Гитлера."

**Нижний текст:** "Мюллер: 'Штирлиц, вы уверены, что у вас достаточно данных?'"

**Подпись:** "Штирлиц: 'Данные есть, но Гитлер — это такой аутлаер, что модель просто не справляется.'"

Надеюсь, вам понравится этот мем!
